### This notebook just gives a basic idea of how to prepare test set and make a baseline submission with few features.

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [3]:
train=pd.read_csv('/kaggle/input/usaids-forecasting/Train.csv')
sd=pd.read_csv('/kaggle/input/usaids-forecasting/service_delivery_site_data.csv')
sub=pd.read_csv('/kaggle/input/usaids-forecasting/SampleSubmission.csv')

In [4]:
train.columns

Index(['year', 'month', 'region', 'district', 'site_code', 'product_code',
       'stock_initial', 'stock_received', 'stock_distributed',
       'stock_adjustment', 'stock_end', 'average_monthly_consumption',
       'stock_stockout_days', 'stock_ordered'],
      dtype='object')

In [6]:
train.drop(columns={'stock_initial', 'stock_received','stock_adjustment', 
                    'stock_end', 'average_monthly_consumption',
                    'stock_stockout_days', 'stock_ordered'},inplace=True)

#### Creating Test Dataset

In [8]:
test=pd.DataFrame(sub.ID.str.split('X',3).tolist(), columns = ['year','month','site_code','product_code'])
for col in test.columns:
    test[col] = test[col].str.strip()
column=['year','month'] 
for col in column:
    test[col]=test[col].astype(int)
test=pd.merge(test,sd,on='site_code',how='left')
test.rename(columns={'site_region':'region','site_district':'district'},inplace=True)
test=test[['year', 'month', 'region','district','site_code', 'product_code']]

In [9]:
train.head(2).append(test.head(2))

,year,month,region,district,site_code,product_code,stock_distributed
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,21.0
1,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27132,3.0
0,2019,7,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS21126,NaN
1,2019,7,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,NaN


#### Concating

In [10]:
train['train_or_test']='train'
test['train_or_test']='test'
df=pd.concat([train,test])

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in ['region','district','site_code','product_code']:
    df[col]=  df[col].astype('str')
    df[col]= le.fit_transform(df[col]) 

In [14]:
train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]
train.drop(columns={'train_or_test'},axis=1,inplace=True)
test.drop(columns={'train_or_test'},axis=1,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
len(test),len(sub)

(3089, 3089)

In [16]:
x=train.drop(columns={'stock_distributed'})
y=train.loc[:,['stock_distributed']]
del test['stock_distributed']

In [17]:
%%time

from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=100)

model.fit(x,y)

CPU times: user 4.25 s, sys: 141 ms, total: 4.4 s
Wall time: 2.32 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
%%time
pred=model.predict(test)

CPU times: user 33.7 ms, sys: 3.08 ms, total: 36.8 ms
Wall time: 11.6 ms


In [19]:
sub['prediction']=np.abs(pred)

In [20]:
sub.to_csv('xgbbaseline.csv',index=False)